In [1]:
#Obtener el Top 200 semanal para un país en particular 
#Test 1: Japon 
#Instalar libreria de Spotify

In [2]:
pip install spotipy --upgrade

Requirement already up-to-date: spotipy in /usr/local/lib/python3.7/dist-packages (2.17.1)


In [3]:
#Credenciales Spotify
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="cc9a7bb857eb4282b846a1fe077003c4",
                                                           client_secret="817188bf767444ed8f6ca6f4080db1af"))

In [4]:
#Funciones para extraer informacion de Spotify 
def get_artist(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        return items[0]
    else:
        return None


def get_artist_album_ids(name):
	artist = get_artist(name)
	results = sp.artist_albums(artist['id'], album_type='album', country='US')
	albums = results['items']
	album_ids = []
	for album in albums:
		album_ids.append(album['id'])

	return album_ids
def get_artist_features(name, feature='valence'):
	# features: valence (default), acousticness, danceability,
	#           energy, instrumentalness, speechiness, & tempo
	album_ids = get_artist_album_ids(name)
	features = []
	for album_id in album_ids:
		results = sp.album_tracks(album_id)
		tracks = results['items']

		for track in tracks:
			audioFeatures = sp.audio_features(track['id'])
			features.append(audioFeatures[0][feature])

	return features


In [8]:
#Funciones para obtener el Top200
import io
import pandas as pd
import requests
import time
import tqdm


def week_dates(date, weekday=0):
    week_start = date - pd.DateOffset(weekday=weekday, weeks=1)
    week_end = date + pd.DateOffset(weekday=weekday, weeks=0)
    return week_start, week_end


def get_chart(date, region='en', freq='weekly', chart='top200'):
    chart = 'regional' if chart == 'top200' else 'viral'
    date = pd.to_datetime(date)
    if date.year < 2017:
        raise ValueError('No chart data available from before 2017')
    if freq == 'weekly':
        start, end = week_dates(date, weekday=4)
        date = f'{start.date()}--{end.date()}'
    else:
        date = f'{date.date()}'
    url = f'https://spotifycharts.com/{chart}/{region}/{freq}/{date}/download'
    data = io.StringIO(requests.get(url).text)
    try:
        df = pd.read_csv(data, skiprows=1) # Fix Spotify's Note
    except pd.errors.ParserError:
        df = None
        print(data)
    return df
    #return url


def get_charts(start, end, region='en', freq='daily', chart='top200', sleep=1):
    sample = 'D' if freq == 'daily' else 'W'
    dfs = []
    for date in tqdm.tqdm(pd.date_range(start=start, end=end, freq=sample)):
        df = get_chart(date, region=region, freq=freq, chart=chart)
        if df is not None:
            df['date'] = date
            dfs.append(df)
            time.sleep(sleep)
    return pd.concat(dfs)

In [9]:
paises = ['Japan']

In [10]:
df = get_chart('2021-03-11', region='jp')

In [8]:
columns_names = df.columns.values
columns_names

array(['Position', 'Track Name', 'Artist', 'Streams', 'URL'], dtype=object)

In [9]:
Artistas = df['Artist'].values
Canciones = df['Track Name'].values
print(len(Artistas), len(Canciones))
print(Canciones[199], Artistas[199])

200 200
Hold Me While You Wait Lewis Capaldi


In [10]:
positividad = []
for i in range(len(Artistas)):
  query = Canciones[i] + '&' + Artistas[i]
  results = sp.search(q=query , type= 'track', limit=3, offset=0)
  #No todas las canciones tienen positividad identificada, por ello toca 
  #Poner una restriccion sobre la busqueda 
  b =  results['tracks']['items']
  if (len(b) == 0):
    continue
  else:
    a = results['tracks']['items'][0]['id']
    positividad.append(sp.audio_features(a)[0]['valence'])



In [11]:
len(positividad)

160

In [12]:
import numpy as np
positividad= np.array(positividad)
positividad.mean()

0.5228093749999999

In [ ]:
j=16
print(Canciones[j], Artistas[j])
query = Canciones[j] + '&' + Artistas[j]
results = sp.search(q=query , type= 'track', limit=3, offset=0)
len(results['tracks']['items'])
#a = results['tracks']['items'][0]['id']
#sp.audio_features(a)[0]['valence']

かくれんぼ 優里


3

In [ ]:
df

,Position,Track Name,Artist,Streams,URL
0,1,ドライフラワー,優里,2387183,https://open.spotify.com/track/7dH0dpi751EoguD...
1,2,Dynamite,BTS,2036551,https://open.spotify.com/track/4saklk6nie3yiGe...
2,3,夜に駆ける,YOASOBI,1991730,https://open.spotify.com/track/6MCjmGYlw6mQVWR...
3,4,勿忘,Awesome City Club,1921045,https://open.spotify.com/track/0XZyF9lv6diMt4b...
4,5,怪物,YOASOBI,1723922,https://open.spotify.com/track/06XQvnJb53SUYml...
...,...,...,...,...,...
195,196,3月9日,Remioromen,166346,https://open.spotify.com/track/1Alsp6EVIAHTDor...
196,197,give it back,Cö Shu Nie,165685,https://open.spotify.com/track/6aHFdm58mPF4YPJ...
197,198,First Love,Hikaru Utada,165250,https://open.spotify.com/track/39HrUxcvKF3jtLz...
198,199,Love so sweet,ARASHI,164197,https://open.spotify.com/track/5I6JLJqnZICLvPe...
